# Effeciant Yelp API Core 
Nokuthula Mchunu

For this assignment,  will be working with the Yelp API.
Will use the Yelp API to search Durban for a African cuisine .

Extract all of the results from your search and compile them into one dataframe using a for loop.

In [48]:
# import necessary liberies 
# Standard Imports
import numpy as np
import pandas as pd

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [49]:
# Load API Credentials
with open('/Users/noksmchunu/.secret/yelp_api.json') as f:   
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [50]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'PASTA'

In [51]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "noksmchunu/Data/results_in_progress_Durban_cusine.json"
JSON_FILE


'noksmchunu/Data/results_in_progress_Durban_cusine.json'

In [52]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")
    else:
         print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
         folder = os.path.dirname(JSON_FILE)
         ## If JSON_FILE included a folder:
         if len(folder)>0:
             # create the folder
             os.makedirs(folder,exist_ok=True)
             ## Save empty list to start the json file
             with open(JSON_FILE,'w') as f:
                 json.dump([],f)  
    
    

In [53]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


[i] noksmchunu/Data/results_in_progress_Durban_cusine.json not found. Saving empty list to new file.
- 0 previous results found.


530

In [54]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/530 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [55]:
# load final results to dataframe 
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4889,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3194,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
2,yXMZSuLJEpNu24yhKey9XA,forma-pasta-factory-brooklyn-3,Forma Pasta Factory,https://s3-media1.fl.yelpcdn.com/bphoto/6LEtDQ...,False,https://www.yelp.com/biz/forma-pasta-factory-b...,128,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.686023409875474, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '5 Greene Ave', 'address2': None,...",+13477991333,(347) 799-1333,2800.607052
3,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1424,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.718554, 'longitude': -73.997447}","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653
4,hdiuRS9sVZSMReZm4oV5SA,da-andrea-new-york,Da Andrea,https://s3-media1.fl.yelpcdn.com/bphoto/OciLdd...,False,https://www.yelp.com/biz/da-andrea-new-york?ad...,2097,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.736218, 'longitude': -73.99597}","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '35 W 13th St', 'address2': '', '...",+12123671979,(212) 367-1979,3424.729337


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,SPeYZneKj9QUt34XNQdrCg,chef-lonnie-dyner-brooklyn-4,Chef Lonnie Dyner,https://s3-media2.fl.yelpcdn.com/bphoto/UThrjB...,False,https://www.yelp.com/biz/chef-lonnie-dyner-bro...,2,"[{'alias': 'catering', 'title': 'Caterers'}, {...",5.0,"{'latitude': 40.67170333862305, 'longitude': -...",[],NaN,"{'address1': None, 'address2': None, 'address3...",+19176644333,(917) 664-4333,5954.050056
996,aPn2_EtjfHduXEmEBNxuFA,rosas-pizza-brooklyn,Rosa's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/TtcGEK...,False,https://www.yelp.com/biz/rosas-pizza-brooklyn?...,196,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.7139499, 'longitude': -73.95563}","[pickup, delivery]",$,"{'address1': '374 Metropolitan Ave', 'address2...",+17189632378,(718) 963-2378,3387.060216
997,byijh-1_NEYYJIXG8uemJg,luigis-restaurant-and-bar-new-hyde-park,Luigi's Restaurant & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/TMU7Xx...,False,https://www.yelp.com/biz/luigis-restaurant-and...,908,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 40.7493936095477, 'longitude': -7...",[delivery],$$,"{'address1': '265-21 Union Tpke', 'address2': ...",+17183477136,(718) 347-7136,24641.838083
998,LvcvCQUYuZttX0e7wiICVA,americas-finest-deli-new-york,Americas Finest Deli,https://s3-media1.fl.yelpcdn.com/bphoto/tmcGIc...,False,https://www.yelp.com/biz/americas-finest-deli-...,64,"[{'alias': 'delis', 'title': 'Delis'}]",3.5,"{'latitude': 40.7076443871245, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '46 Trinity Pl', 'address2': None...",+12125090930,(212) 509-0930,1614.836453
999,XcfZqLrXdABSZLViQ34VTw,chelsea-ristorante-new-york,Chelsea Ristorante,https://s3-media2.fl.yelpcdn.com/bphoto/Fqs1MI...,False,https://www.yelp.com/biz/chelsea-ristorante-ne...,224,"[{'alias': 'italian', 'title': 'Italian'}]",3.5,"{'latitude': 40.7405719, 'longitude': -74.0017...","[pickup, delivery]",$$,"{'address1': '108 8th Ave', 'address2': '', 'a...",+12129247786,(212) 924-7786,3962.824349
